In [1]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%cd /content/drive/Othercomputers/Computer1/project-m2-2024-fantasticfour

/content/drive/Othercomputers/Computer1/project-m2-2024-fantasticfour


In [ ]:
!pip install datasets==2.17.1
!pip install trl
!pip install peft
!pip install flash-attn --no-build-isolation


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.w

In [6]:
# Imports
import pandas as pd
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from datasets import load_from_disk
from datasets import Dataset
from datasets import concatenate_datasets
from peft import PeftModel
from trl import DPOTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from peft import LoraConfig, PeftConfig
from transformers.integrations import TensorBoardCallback

In [3]:
# Create a DataFrame with your data
data = {
    'Title': ['Document 1', 'Document 2', 'Document 3'],
    'Text': ['Garik is a famous AMV-maker, making horror videos.', 'Baptiste is a member of the Fantastic Four team, he is THE GOAT.', 'Rami is the expert of github in EPFL.']
}

df = pd.DataFrame(data)

# Convert df to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [3]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizerFast

# Load DPR context encoder and tokenizer
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

C:\Users\41794\anaconda3\envs\assMNLP\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identica

In [5]:
# Function to encode documents
def embed(documents):
    inputs = ctx_tokenizer(
        documents["Title"], documents["Text"], truncation=True, padding="longest", return_tensors="pt"
    )
    embeddings = ctx_encoder(**inputs).pooler_output
    return {"embeddings": embeddings.detach().cpu().numpy()}

# Encode documents
dataset = dataset.map(embed, batched=True)

# Save the dataset with embeddings
dataset.save_to_disk("my_dpr_dataset")

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
import faiss
from datasets import load_from_disk

# Load the dataset
dataset = load_from_disk("encoded_dataset")

# Convert embeddings to numpy array to determine the dimension
embeddings = np.array(dataset["embeddings"])
d = embeddings.shape[-1] # Dimension of the embeddings

# Create FAISS index
index = faiss.IndexHNSWFlat(d, 128, faiss.METRIC_INNER_PRODUCT)
dataset.add_faiss_index("embeddings", custom_index=index)

# Save the FAISS index
dataset.get_index("embeddings").save("my_dpr_index.faiss")


  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import numpy as np

# Load DPR question encoder and tokenizer
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

# Encode the question
question = "When computing PageRank iteratively, the computation ends when:?\n\nOptions:\nA. The norm of the difference of rank vectors of two subsequent iterations falls below a predefined threshold\nB. The difference among the eigenvalues of two subsequent iterations falls below a predefined threshold\nC. All nodes of the graph have been visited at least once\nD. The probability of visiting an unseen node falls below a predefined threshold\n\nAnswer:"
inputs = question_tokenizer(question, return_tensors="pt")
question_embedding = question_encoder(**inputs).pooler_output.detach().cpu().numpy()

# Load the dataset and FAISS index
dataset = load_from_disk("encoded_dataset")
dataset["train"].load_faiss_index("embeddings", "encoded_dataset_index.faiss")

# Perform the retrieval
scores, retrieved_examples = dataset["train"].get_nearest_examples("embeddings", np.array(question_embedding), k=5)

# Print the retrieved documents
for i, example in enumerate(retrieved_examples["text"]):
    print(f"Document {i + 1}: {example}")


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Document 1: Topological ordering algorithm?
Topological ordering algorithm is used to find the linear ordering of vertices (or nodes) in a directed acyclic graph (DAG). Here is the algorithm:

1. Determine the in-degree of each vertex (the number of incoming edges to each vertex) in the DAG.
2. Pick any vertex with an in-degree of 0 and add it to the ordered list of vertices.
3. Delete the chosen vertex and all of its outgoing edges from the graph.
4. Update the in-degree of the remaining vertices in the graph.
5. Repeat steps 2-4 until all vertices have been added to the ordered list.

At the end of the algorithm, if the DAG has no cycles, then the list of vertices represents a valid topological ordering. However, if the DAG has cycles, it is impossible to find a valid topological ordering since the cycles cause a circular dependency. In such cases, the algorithm will terminate before ordering all vertices.
Document 2: Which condition is an interval colouring?
An interval coloring of 

In [89]:
# Load GPT-2 model and tokenizer
model_dir = '../checkpoints/CheatSheet DPO augmented'

tokenizer = AutoTokenizer.from_pretrained(model_dir)

# take the same peft_config as for the SFTTrainer
peft_config = PeftConfig.from_pretrained(model_dir)

# Load model
base_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-large")
model = PeftModel.from_pretrained(base_model, model_dir, is_trainable = True)

In [90]:
# Ensure model is in evaluation mode and move to the appropriate device
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assume retrieved_examples contains the top 2 retrieved documents
retrieved_contexts = retrieved_examples["Text"]
context = " ".join(retrieved_contexts)

# Prepare the input text by concatenating the question with the context
input_text = context + " " + question + " "

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding="max_length", max_length=100)

# Move inputs to the appropriate device
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate the output
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=256, # You can adjust the max_length as needed
        num_beams=3,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

# Decode the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract the answer from the generated text
print("Question:", question)
print("Generated Answer:", generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question:  Who is member of the fantastic four team ?
Generated Answer: Baptiste is a member of the Fantastic Four team, he is THE GOAT.  Who is member of the fantastic four team? The Fantastic four are a group of super-heroes that have been around since the 1940's. They are the most popular superhero team in the world. The Fantastic 4 are also known as the Four Horsemen of Doom.

The team consists of Reed Richards, Ben Grimm, Johnny Storm, and Sue Storm. Reed is the leader and Ben is his sidekick. Johnny is Reed's best friend. Sue is Ben's girlfriend. Ben and Johnny are good friends, but they don't like each other very much. In the comics, they have a rivalry with the Human Torch, who is also part of their team. This rivalry is usually resolved with a fight between the two teams, which usually ends with Reed being the victor. However, in some stories, the
